Run imports
===

In [321]:
import wget
import zipfile
import os
from subprocess import call
from shutil import move
import gzip
import xml.etree.ElementTree as ET
from collections import Counter
import operator
import csv
import re

Pull files from web
===

In [2]:
filename = wget.download("http://www.drugbank.ca/system/downloads/current/drugbank.xml.zip")

In [1]:
zfile = zipfile.ZipFile(filename)
zfile.extract('drugbank.xml', 'data/new.drugbank.xml')
os.remove(filename)

NameError: name 'zipfile' is not defined

In [2]:
def extract(file):
    with gzip.open(file, 'rb') as rf:
        with open('data/' + file.rsplit('.',1)[0] + '.human', 'w') as wf:
            for line in rf:
                line_ascii = line.decode('utf-8')
                species = line_ascii.split()[0]
                if species == '9606':  # Grab human only
                    wf.write(line_ascii)

In [10]:
g2a_filename = wget.download("ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2accession.gz")
gi_filename = wget.download("ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene_info.gz")
# g2up_filename = wget.download("ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene_refseq_uniprotkb_collab.gz")
extract("gene_info.gz")
extract("gene2accession.gz")
# extract("gene_refseq_uniprotkb_collab.gz")
os.remove(g2a_filename)
os.remove(gi_filename)
# os.remove(g2up_filename)

Process drugbank xml
===

Check if update is needed
---

In [4]:
compare = os.path.isfile('data/drugbank.xml')
if compare:
    # check if the new file hash is the same as the old file. If so, run updates.
    pass
else:
    # no old file to compare against. run updates.
    pass
# move new file to old file

Load gene annotation resources
---

In [331]:
symbol_to_info = dict()
hgnc_id_to_info = dict()
entrez_to_info = dict()
sources = set()
with open('data/gene_info.human') as f:
    c = csv.reader(f, delimiter='\t')
    for line in c:
        if line[0] != '9606':
            continue
        gene_symbol = line[2]
        entrez_id = line[1]
        symbol_to_info[gene_symbol] = {'Entrez': entrez_id,
                                       'Symbol': gene_symbol}
        if line[5] == '-':
            continue
        synonyms = line[5].split('|')
        for synonym in synonyms:
            (source, accession) = synonym.split(':', 1)
            symbol_to_info[gene_symbol][source] = accession
            sources.add(source)
        if 'HGNC' in symbol_to_info[gene_symbol]:
            hgnc_id_to_info[symbol_to_info[gene_symbol]['HGNC']] = symbol_to_info[gene_symbol]
        entrez_to_info[entrez_id] = symbol_to_info[gene_symbol]

In [322]:
uniprot_to_entrez = dict()
r = re.compile(r'[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}')
    # regex from: http://www.uniprot.org/help/accession_numbers
with open('data/gene2accession.human') as f:
    c = csv.reader(f, delimiter='\t')
    for line in c:
        if line[0] != '9606':
            continue
        uniprot_id = line[5].split('.',1)[0]
        if not r.match(uniprot_id):
            continue
        entrez_id = line[1]
        uniprot_to_entrez[uniprot_id] = entrez_id

In [323]:
len(uniprot_to_entrez)

19850

Parse XML
---

In [51]:
ns = {'entry': 'http://www.drugbank.ca'}

def elements(x):
    c = Counter()
    for elem in x:
        c[elem.tag[len(ns['entry'])+2:]] += 1
    return c

In [70]:
tree = ET.parse('data/old.drugbank.xml')
drugbank = tree.getroot()

In [161]:
drugs = drugbank.findall('entry:drug', ns)
drug = drugs[182] # arbitrary

In [349]:
interactions = dict()
drug_info = dict()
uniprot_fail = 0
hgnc_fail = 0
no_info = 0
total = 0
for drug in drugs:
    drug_id = drug.find('entry:drugbank-id', ns).text
    drug_name = drug.find('entry:name', ns).text
    
    synonyms = drug.find('entry:synonyms', ns)
    drug_synonyms = set()
    for synonym in synonyms:
        language = synonym.get('language')
        if language == '' or language == 'English':
            drug_synonyms.add(synonym.text)
    drug_cas_number = drug.find('entry:cas-number',ns).text
    drug_brands = set()
    for product in drug.find('entry:products', ns):
        drug_brands.add(product.find('entry:name', ns).text)
    for int_brand in drug.find('entry:international-brands', ns):
        drug_brands.add(product.find('entry:name', ns).text)
    drug_type = drug.get('type')
    drug_groups = set()
    for group in drug.find('entry:groups', ns):
        drug_groups.add(group.text)
    drug_categories = set()
    for category in drug.find('entry:categories', ns):
        drug_categories.add(category.find('entry:category', ns).text)

    # Left off here, looking at gene_id onward in drug_bank_row.rb
    targets = drug.find('entry:targets', ns)
    if len(targets) == 0:
        continue
    drug_info[drug_id] = (drug_name, drug_synonyms, drug_cas_number, drug_brands,
                          drug_type, drug_groups, drug_categories)
    for target in targets:
        organism = target.find('entry:organism', ns).text
        if organism != 'Human':
            continue
        gene_id = target.find('entry:id',ns).text
        known_action = target.find('entry:known-action', ns).text
        target_actions = set()
        for action in target.find('entry:actions', ns):
            target_actions.add(action.text)
        gene_symbol = hgnc_gene_symbol = uniprot_id = entrez_id = ensembl_id = None
        polypeptide = target.find('entry:polypeptide',ns)
        synonyms = None
        if polypeptide is not None:
            gene_symbol = polypeptide.find('entry:gene-name', ns).text
            for identifier in polypeptide.find('entry:external-identifiers',ns):
                if identifier.find('entry:resource',ns).text == 'HUGO Gene Nomenclature Committee (HGNC)':
                    hgnc_gene_symbol = identifier.find('entry:identifier',ns).text
                    # Some identifiers are incorrectly labeled as 'GNC:nnn'
                    r = re.compile(r'^\d+$')
                    if hgnc_gene_symbol.startswith('GNC:'):
                        hgnc_gene_symbol = 'H' + hgnc_gene_symbol
                    elif r.match(hgnc_gene_symbol):
                        hgnc_gene_symbol = 'HGNC:' + hgnc_gene_symbol
                    try:
                        synonyms = hgnc_id_to_info[hgnc_gene_symbol]
                    except:
                        hgnc_fail += 1
                if identifier.find('entry:resource',ns).text == 'UniProtKB':
                    uniprot_id = identifier.find('entry:identifier',ns).text
                    if not synonyms:
                        try:
                            entrez_id = uniprot_to_entrez[uniprot_id]
                            synonyms = entrez_to_info[entrez_id]
                        except KeyError:
                            uniprot_fail += 1
        if not synonyms:
            try:
                synonyms = symbol_to_info[gene_symbol]
            except KeyError:
                no_info += 1
            else:
                entrez_id = synonyms['Entrez']
                ensembl_id = synonyms['Ensembl']
        interaction_tuple = (gene_id, known_action, target_actions, gene_symbol,
                             uniprot_id, entrez_id, ensembl_id)
        total += 1
        try:
            interactions[drug_id].append(interaction_tuple)
        except KeyError:
            interactions[drug_id] = [interaction_tuple,]
    

190

In [345]:
hgnc_gene_symbol

'GNC:668'

In [327]:
elements(category)

Counter({'mesh-id': 1, 'category': 1})

In [332]:
sources

{'Ensembl', 'HGNC', 'HPRD', 'IMGT/GENE-DB', 'MIM', 'Vega', 'miRBase'}